In [122]:
from bs4 import BeautifulSoup
import requests
import re
import csv

root = "https://www.unikorea.go.kr/"
web_link = list()
result = list()

# 통일부 내의 복지 페이지 링크를 모두 가져옴
def enum_links(html):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("#content > div.content-body > div.div-minwon-form > div > nav > div > ul") # 복지링크 테이블
    herfs = re.findall('(?<=href=").*?(?=[\"])', str(links))
    
    for herf in herfs:
        url = urljoin(root, herf)
        web_link.append(url)
    web_link[0] = "https://www.unikorea.go.kr/unikorea/minwon/Info/fundstep/NKDefectors/achievement/"
    
if __name__ == "__main__":
    url = "https://www.unikorea.go.kr/unikorea/minwon/Info/fundstep/NKDefectors/achievement/"
    req = requests.get(url)
    html = req.text
    enum_links(html)
    
    for link in web_link:
        req = requests.get(link)
        html = req.text
        soup = BeautifulSoup(html, "html.parser")
        title = [x.get_text() for x in soup.select("#wrap > div.container.sub.sub019814 > div.content-wrap > div > h2")][0]
        target = "탈북민"
        contents = soup.select("#content > div.content-body > div.div-minwon-form > div > div.tab2-content1 > div > section > div > div")
        discription = list()
        for content in contents:
            content = str(content)            
            while content.find('<dt>') != -1:
                t_start, t_end = content.find('<dt>')+4, content.find('</dt>')
                d_start, d_end = content.find('<dd>')+4, content.find('</dd>')
                disc = re.sub('<.+?>', '', content[d_start:d_end], 0, re.I|re.S) #태그 제거
                disc = re.sub('&nbsp;| |\t|\r', '', disc.strip()) #공백제거
                disc = re.sub('[\\n]+', ',', disc) # 개행 처리

                discription.append([ content[t_start:t_end]+ ':' + disc ])
                content = content[d_end+4:]
            result.append([title, target, discription, link])
            dscription = []

    csv_file = open("./unikorea.csv", "w", newline="", encoding='UTF8')
    csv_writer = csv.writer(csv_file)
    for row in result:
        csv_writer.writerow(row)
    csv_file.close()